### Updated on 11/05/2021
<br>
<h4>Sequential Version </h4> 
<br>
This Notebook implements the 3 party Quantum Conference Protocol

In [1]:
from qiskit import *
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
import matplotlib.pyplot as plotter
import numpy as np
from IPython.display import display, Math, Latex, Image
%matplotlib inline

#### Step 1: Generation of Message and preparation or random permutation
<br>
1. Here qa, qb, qc are the random of permutations of Qubits of Alice, Bob and Charlie respectively
<br>2. delta_m is the number used for checking the error in the channel 

In [2]:
num_bits = 32

key_ini = np.random.randint(2, size = num_bits)

alice_mess_ini = np.random.randint(2, size = num_bits)
bob_mess_ini = np.random.randint(2, size = num_bits)
charlie_mess_ini = np.random.randint(2, size = num_bits)

qa_ini = np.random.choice(num_bits, size = num_bits, replace = False)
qb_ini = np.random.choice(num_bits, size = num_bits, replace = False)
qc_ini = np.random.choice(num_bits, size = num_bits, replace = False)

In [3]:
delta_m = 4

common_idx = np.sort(np.random.choice(num_bits, size = delta_m, replace = False))
print(common_idx)

[ 4  8 11 26]


In [4]:
def encode_message(key_bit, message_bit, qubit, circuit):
    if message_bit == 1:
        circuit.x(qubit)
    if key_bit == 1:
        circuit.h(qubit)
    return circuit

def run_circuit(qc, shot):
    backend = Aer.get_backend('qasm_simulator') # we choose the simulator as our backend
    result = execute(qc, backend, shots = shot).result() # we run the simulation
    counts = result.get_counts() # we get the counts
    return counts

def count_to_str(counts):
    r = counts.items()
    for res, shots in r:
        result = str(res)
    return result

#### Checking of Error in the Channel 

In [5]:
result = []

for i in range(delta_m):
    
    a = QuantumRegister(1)
    b = QuantumRegister(1)
    c = QuantumRegister(1)
    ufp = ClassicalRegister(3)
    circuit = QuantumCircuit(a, b, c, ufp)
    
    idx = common_idx[i]
    
    #print(idx)
    
    a_mess = alice_mess_ini[idx]
    b_mess = bob_mess_ini[idx]
    c_mess = charlie_mess_ini[idx]
    k = key_ini[idx]
    
    #print(a_mess, b_mess, c_mess, k)
    
    encode_message(k, a_mess, a[0], circuit)
    encode_message(k, b_mess, b[0], circuit)
    encode_message(k, c_mess, c[0], circuit)
    
    circuit.barrier()
    
    #Eaves Dropper could be introduced here so to check the exact security of the protocol with the threshold
    
    circuit.barrier()
    
    if(k == 1):
        circuit.h([a[0], b[0], c[0]])
    
    circuit.measure([a[0], b[0], c[0]], ufp)
    
    counts = run_circuit(circuit, 1)
    mess = count_to_str(counts)
    mess = mess[::-1]
    
    for j in range(len(mess)):
        result.append((int)(mess[j]))

result = np.array(result)

In [6]:
flag = 1

for i in range(delta_m):
    idx = common_idx[i]
    a = result[3*i]
    b = result[3*i+1]
    c = result[3*i+2]
    
    if a != alice_mess_ini[idx] or b != bob_mess_ini[idx] or c != charlie_mess_ini[idx]:
        flag = 0

if flag == 0:
    print("Security Checking Failed !")
else:
    print("Security Checking Passed !")

Security Checking Passed !


#### Removing the indexes which were used to check the security

In [7]:
key = []
alice_mess = []
bob_mess = []
charlie_mess = []

for i in range(num_bits):
    if i not in common_idx:
        key.append(key_ini[i])
        alice_mess.append(alice_mess_ini[i])
        bob_mess.append(bob_mess_ini[i])
        charlie_mess.append(charlie_mess_ini[i])
        
key = np.array(key)
alice_mess = np.array(alice_mess)
bob_mess = np.array(bob_mess)
charlie_mess = np.array(charlie_mess)
new_num_bits = len(charlie_mess)

In [8]:
final_result = []

for i in range(new_num_bits):
    
    a = QuantumRegister(1)
    b = QuantumRegister(1)
    c = QuantumRegister(1)
    ufp = ClassicalRegister(3)
    circuit = QuantumCircuit(a, b, c, ufp)
    
    a_mess = alice_mess[i]
    b_mess = bob_mess[i]
    c_mess = charlie_mess[i]
    k = key[i]
    #print(a_mess, b_mess, c_mess, k)
    
    encode_message(k, a_mess, a[0], circuit)
    encode_message(k, b_mess, b[0], circuit)
    encode_message(k, c_mess, c[0], circuit)
    
    circuit.barrier()
    
    circuit.cx(a[0], b[0])
    circuit.cx(a[0], c[0])
    circuit.h(a[0])
    
    circuit.barrier()
    
    circuit.measure([a[0], b[0], c[0]], ufp)
    
    counts = run_circuit(circuit, 1)
    mess = count_to_str(counts)
    mess = mess[::-1]
    
    #print(mess)
    for j in range(len(mess)):
        final_result.append((int)(mess[j]))

final_result = np.array(final_result)

#### Decryption of the Message
<br>Now to decrypt the messages we have to look at the key bits. <br>
1. If key is 0 then one person directly knows the messages of other two 
<br>
2. If the key is 1 then we have to apply the method of Xor computation

In [9]:
dec_a_for_b = np.empty(shape = new_num_bits, dtype = int)
dec_c_for_b = np.empty(shape = new_num_bits, dtype = int)

dec_a_for_c = np.empty(shape = new_num_bits, dtype = int)
dec_b_for_c = np.empty(shape = new_num_bits, dtype = int)

dec_b_for_a = np.empty(shape = new_num_bits, dtype = int)
dec_c_for_a = np.empty(shape = new_num_bits, dtype = int)

one_idx = []
alice_xor = []
bob_xor = []
charlie_xor = []

for i in range(new_num_bits):
    
    if key[i] == 0:
        
        a = (int)(final_result[3*i])
        b = (int)(final_result[3*i + 1])
        c = (int)(final_result[3*i + 2])
        if alice_mess[i] == 0:
            dec_b_for_a[i] = b 
            dec_c_for_a[i] = c
        else:
            dec_b_for_a[i] = 1 - b 
            dec_c_for_a[i] = 1 - c
        
        a = 0
        
        if bob_mess[i] == b:
            dec_a_for_b[i] = a
            dec_c_for_b[i] = c
        else :
            dec_a_for_b[i] = 1 - a
            dec_c_for_b[i] = 1 - c
        
        if charlie_mess[i] == c:
            dec_a_for_c[i] = a
            dec_b_for_c[i] = b
        else :
            dec_a_for_c[i] = 1 - a
            dec_b_for_c[i] = 1 - b
        
#         print(key[i], a, b, c)    
#         print(alice_mess[i], bob_mess[i], charlie_mess[i])
#         print(dec_a_for_c[i], dec_b_for_c[i], dec_c_for_b[i])
        
    else:
        one_idx.append(i)
        #Here where the key bit is 1 the XOR value of other two are stored     
        if alice_mess[i] == final_result[3*i]:
            alice_xor.append(0)
        else:
            alice_xor.append(1)
        if bob_mess[i] == final_result[3*i]:
            bob_xor.append(0)
        else:
            bob_xor.append(1)
        if charlie_mess[i] == final_result[3*i]:
            charlie_xor.append(0)
        else:
            charlie_xor.append(1)

Now the steps for XOR computation will be done first new ordered set of qubits are prepared and then some decoy photons are added <br>
Now here we could have applied random permutations but it will of no use because at last it is only going to get un-permuted. <br>

In [10]:
num_decoy = 5

decoy_a_base = np.random.randint(2, size = num_decoy)
decoy_a_bit = np.random.randint(2, size = num_decoy)

decoy_b_base = np.random.randint(2, size = num_decoy)
decoy_b_bit = np.random.randint(2, size = num_decoy)

decoy_c_base = np.random.randint(2, size = num_decoy)
decoy_c_bit = np.random.randint(2, size = num_decoy)

num_xor_total = num_decoy + len(one_idx)

flag = 1

Now for a channel first the security for the decoy bit will be checked then only a person will proceed with others qubit for message decoding <br>
Now here we are considering Alice sends her qubits to Bob, Bob sends his to Charlie and Charlie back to Alice 
<br> Here flag is used to just to keep a check if anytime security is breached or not 

In [11]:
# First the channel correctness is checked by Bob for Alice Qubits 

decoded_security = []

for i in range(num_decoy):
    
    a = QuantumRegister(1)
    meas = ClassicalRegister(1)
    circuit = QuantumCircuit(a, meas)
    
    if decoy_a_bit[i] == 1:
        circuit.x(a[0])
    if decoy_a_base[i] == 1:
        circuit.h(a[0])
    
    circuit.barrier()
    
    #Eaves Dropper possible actions here
    
    circuit.barrier()
    
    # Now as Bob knows the bases 
    if decoy_a_base[i] == 1:
        circuit.h(a[0])
        
    circuit.measure(a, meas)
    counts = run_circuit(circuit, 1)
    
    res = count_to_str(counts)
    decoded_security.append((int)(res))
    
decoded_security = np.array(decoded_security)

comparison = decoded_security == decoy_a_bit
equal_arrays = comparison.all()

if equal_arrays == True:
    print("Security Checking for Alice to Bob Channel Passed !")
else:
    print("Security Checking for Alice to Bob Channel Failed !")
    flag = 0

Security Checking for Alice to Bob Channel Passed !


In [12]:
# Now the Security Checking for Bob To Charlie is conducted by Charlie

decoded_security = []

for i in range(num_decoy):
    
    b = QuantumRegister(1)
    meas = ClassicalRegister(1)
    circuit = QuantumCircuit(b, meas)
    
    if decoy_b_bit[i] == 1:
        circuit.x(b[0])
    if decoy_b_base[i] == 1:
        circuit.h(b[0])
    
    circuit.barrier()
    #Eaves Dropper possible actions here
    circuit.barrier()
    
    # Now as Charlie knows the bases 
    if decoy_b_base[i] == 1:
        circuit.h(b[0])
        
    circuit.measure(b, meas)
    counts = run_circuit(circuit, 1)
    
    res = count_to_str(counts)
    decoded_security.append((int)(res))
    
decoded_security = np.array(decoded_security)

comparison = decoded_security == decoy_b_bit
equal_arrays = comparison.all()

if equal_arrays == True:
    print("Security Checking for Bob to Charlie Channel Passed !")
else:
    print("Security Checking for Bob to Charlie Channel Failed !")
    flag = 0

Security Checking for Bob to Charlie Channel Passed !


In [13]:
# Now the Security Checking for Charlie to Alice is conducted by Alice
decoded_security = []
for i in range(num_decoy):
    c = QuantumRegister(1)
    meas = ClassicalRegister(1)
    circuit = QuantumCircuit(c, meas)
    
    if decoy_c_bit[i] == 1:
        circuit.x(c[0])
    if decoy_c_base[i] == 1:
        circuit.h(c[0])
    
    circuit.barrier()
    #Eaves Dropper possible actions here
    circuit.barrier()
    
    # Now as Alice knows the bases 
    if decoy_c_base[i] == 1:
        circuit.h(c[0])
        
    circuit.measure(c, meas)
    counts = run_circuit(circuit, 1)
    
    res = count_to_str(counts)
    decoded_security.append((int)(res))
    
decoded_security = np.array(decoded_security)

comparison = decoded_security == decoy_c_bit
equal_arrays = comparison.all()

if equal_arrays == True:
    print("Security Checking for Charlie to Alice Channel Passed !")
else:
    print("Security Checking for Charlie to Alice Channel Failed !")
    flag = 0

Security Checking for Charlie to Alice Channel Passed !


#### Xor Computation starts here 

In [14]:
num_xor = len(one_idx)

#Now as the channels were secure now the message could be decoded 
#Here bob is decoding the messages for himself

for i in range(num_xor):
    
    circuit = QuantumCircuit(1, 1)
    b_xor = bob_xor[i]
    idx = one_idx[i]
    even = idx%2
    
    if alice_mess[idx] == 1:
        circuit.x(0)
    if even == 1:
        circuit.h(0)
        
    circuit.barrier()
    
    if even == 1:
        circuit.h(0)
    
    circuit.measure(0, 0)
    
    count = run_circuit(circuit,1)
    res = count_to_str(count)
    #print(res)
    dec_a_for_b[idx] = (int)(res)
    dec_c_for_b[idx] = b_xor ^ (int)(res)

In [15]:
#Here charlie is decoding the messages for himself and that got from bob

for i in range(num_xor):
    
    circuit = QuantumCircuit(1, 1)
    c_xor = charlie_xor[i]
    idx = one_idx[i]
    even = idx%2
    
    if bob_mess[idx] == 1:
        circuit.x(0)
    if even == 1:
        circuit.h(0)
        
    circuit.barrier()
    
    if even == 1:
        circuit.h(0)
    
    circuit.measure(0, 0)
    
    count = run_circuit(circuit,1)
    res = count_to_str(count)
    #print(res)
    dec_b_for_c[idx] = (int)(res)
    dec_a_for_c[idx] = c_xor ^ (int)(res)

In [16]:
#Here alice is decoding the messages for herself from the qubits of Charlie

for i in range(num_xor):
    
    circuit = QuantumCircuit(1, 1)
    a_xor = alice_xor[i]
    idx = one_idx[i]
    even = idx%2
    
    if charlie_mess[idx] == 1:
        circuit.x(0)
    if even == 1:
        circuit.h(0)
        
    circuit.barrier()
    
    if even == 1:
        circuit.h(0)
    
    circuit.measure(0, 0)
    
    count = run_circuit(circuit,1)
    res = count_to_str(count)
    #print(res)
    dec_c_for_a[idx] = (int)(res)
    dec_b_for_a[idx] = a_xor ^ (int)(res)

In [17]:
def compare(a1, a2):
    comp = a1 == a2
    equality = comp.all()
    return equality

print(compare(dec_a_for_b, alice_mess))
print(compare(dec_a_for_c, alice_mess))
print(compare(dec_c_for_b, charlie_mess))
print(compare(dec_b_for_c, bob_mess))
print(compare(dec_c_for_a, charlie_mess))
print(compare(dec_b_for_a, bob_mess))

True
True
True
True
True
True
